# 4. Modeling

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

/Users/sedkin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
Location = r'../final_3/clean_data.csv'
df = pd.read_csv(Location,  encoding='utf-8')

In [3]:
df.head()

,article_description,article_pubtime,article_section,article_subtopic,article_tags,article_topic,author_name,content,title
0,If you're having trouble trying to be more inn...,2016-11-11T20:00:15+00:00,Planning for Growth,Growth Opportunities,"[Growth Opportunities, Innovation, Personal Pr...",Planning for Growth,Heather Willems,The Power of Non-Linear ThinkingHeather Willem...,The Power of Non-Linear Thinking
1,Huge growth may be possible for businesses of ...,2016-11-09T22:30:34+00:00,Planning for Growth,Operations,"[Business Expansion, Business Plan, Cash Flow,...",Planning for Growth,Andrew J. Sherman,3 Growth Strategies for Businesses With Limite...,3 Growth Strategies for Businesses With Limite...
2,What sets your business apart from the competi...,2016-11-11T22:30:47+00:00,Getting Customers,Marketing & Sales,"[Advertising, Brainstorming, Branding, Busines...",Getting Customers,Rieva Lesonsky,USP 101: How to Uncover Your Unique Selling Pr...,USP 101: How to Uncover Your Unique Selling Pr...
3,"Organizational changes are rarely fun or easy,...",2016-11-03T23:30:02+00:00,Building Your Team,Company Culture,"[Building Your Team, Business Expansion, Busin...",Building Your Team,David Niu,8 Strategies for Big (or Small) Organizational...,8 Strategies for Big (or Small) Organizational...
4,"When expanding your business, exporting to int...",2016-10-27T19:00:39+00:00,Planning for Growth,Growth Opportunities,"[Growth Opportunities, Importing & Exporting, ...",Planning for Growth,Greg Sandler,Lessons Learned: Four Stories From Experienced...,Lessons Learned: Four Stories From Experienced...


In [4]:
df['article_section'].value_counts()

Building Your Team     26
Getting Customers      26
Planning for Growth    19
Managing Money         12
Name: article_section, dtype: int64

In [5]:
df['article_subtopic'].value_counts()

Customer Relations       12
Leadership               12
Marketing & Sales         9
Company Culture           8
Growth Opportunities      8
Cash Flow                 6
Hiring & HR               6
Strategy                  4
Taxes                     4
Social Media Strategy     3
Research                  3
Operations                3
Financing                 2
Digital Tools             2
Productivity              1
Name: article_subtopic, dtype: int64

In [6]:
#df[df['author_name'] == 'David Niu']

In [7]:
# split data into train and test sets
from sklearn.cross_validation import train_test_split
data_train, data_test = train_test_split(df, test_size=0.33, random_state=42)
len(data_train)

data_train.head()
labels = data_train['article_subtopic']
print "Labels \n", labels.head()

labels_test = data_test['article_subtopic']

Labels 
16                 Strategy
39                    Taxes
62    Social Media Strategy
72              Hiring & HR
7                  Research
Name: article_subtopic, dtype: object


In [8]:
growth_opp = df[df['article_subtopic'] == 'Growth Opportunities']
hiring_hr = df[df['article_subtopic'] == 'Hiring & HR']
print len(hiring_hr)
hiring_labels = pd.Series(["Hiring & HR","Hiring & HR","Hiring & HR","Hiring & HR","Hiring & HR","Hiring & HR"])
len(hiring_labels)

6


6

In [9]:
# vectorize
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_features = 50, 
                             ngram_range=(1, 3), 
                             stop_words='english',
                             binary=True)
X_train_counts = count_vect.fit(data_train['content'])
#X_train_counts.shape
X_train_counts = count_vect.transform(data_train['content'])
names =  count_vect.get_feature_names()
names[115:145]

[]

In [10]:
count_vect.vocabulary_.get(u'business')

7

In [11]:
count_vect.vocabulary_.get(u'employees')

In [12]:
count_vect.vocabulary_.get(u'stock')

In [13]:
count_vect.vocabulary_.get(u'bread')

In [14]:
# vectorize with tf-idf 
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print X_train_tf.shape

names = count_vect.get_feature_names()
names[50:100]

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print X_train_tfidf.shape

(55, 50)
(55, 50)


In [15]:
# use Naive-Bayes Classifier 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, labels)

In [16]:
print labels[0:5]

16                 Strategy
39                    Taxes
62    Social Media Strategy
72              Hiring & HR
7                  Research
Name: article_subtopic, dtype: object


In [17]:
# Test the classifier 
docs_new = ['Business strategy is important.', 
            "Hiring new employees is tough because candidates are hard to find.", 
            "Candidates hire new employees", 
            "Web SEO mobile",
           "Attracting new customers through advertising",
           "Employee benefits",
           "Economic Ups and Downs",
           "Ups and Downs",
           "Customer stuff",
           "mobile",
           "digital",
           "The Power of Non-Linear Thinking",
           "3 Growth Strategies for Businesses With Limited"]
X_new_counts = count_vect.transform(docs_new)

X_new_tf = tf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, category))

'Business strategy is important.' => Leadership
'Hiring new employees is tough because candidates are hard to find.' => Customer Relations
'Candidates hire new employees' => Customer Relations
'Web SEO mobile' => Leadership
'Attracting new customers through advertising' => Customer Relations
'Employee benefits' => Leadership
'Economic Ups and Downs' => Leadership
'Ups and Downs' => Leadership
'Customer stuff' => Leadership
'mobile' => Leadership
'digital' => Leadership
'The Power of Non-Linear Thinking' => Leadership
'3 Growth Strategies for Businesses With Limited' => Customer Relations


In [18]:
# creates a pipeline 
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

In [19]:
text_clf = text_clf.fit(data_train['content'], labels)

In [20]:
# get prediction accuracy using the test set
docs_test = data_test['content']
#print docs_test
predicted = text_clf.predict(docs_test)
#print predicted
np.mean(predicted == labels_test) 
text_clf.score(docs_test, labels_test)

0.035714285714285712

In [21]:
print data_train['article_subtopic'].value_counts()

Leadership               11
Customer Relations        9
Company Culture           6
Marketing & Sales         5
Hiring & HR               4
Strategy                  3
Taxes                     3
Operations                3
Cash Flow                 2
Financing                 2
Research                  2
Growth Opportunities      2
Social Media Strategy     1
Digital Tools             1
Productivity              1
Name: article_subtopic, dtype: int64


My classifier is only bucketing documents into two categories, despite feeding it several categories. This might be because there are a lot of "Leadership" and "Customer Relations" articles and not as many of the other categories.

Let's try it again using categories that all have the same number of observations - to see if that was causing it.

In [22]:
categories = ['Cash Flow', 'Financing', 'Research', 'Growth Opportunities', 'Strategy','Taxes', 'Operations']
# categories = ['Building Your Team', 'Getting Customers', 'Planning for Growth']
train_dropped = data_train[data_train['article_subtopic'].isin(categories)]
train_dropped.head(2)

,article_description,article_pubtime,article_section,article_subtopic,article_tags,article_topic,author_name,content,title
16,Small businesses can be just as volatile as th...,2016-08-23T22:30:24+00:00,Planning for Growth,Strategy,"[Accounts Receivable/Payable, Building Your Te...",Planning for Growth,Julie Bawden Davis,Economic Ups and Downs: How to Help Prepare Yo...,Economic Ups and Downs: How to Help Prepare Yo...
39,Being as tax-savvy as possible can help you—an...,2016-09-29T17:00:01+00:00,Managing Money,Taxes,"[Cash Flow, Managing Money, Tax Deductions, Ta...",Managing Money,John Suh,4 Things Every Business Owner Should Know Abou...,4 Things Every Business Owner Should Know Abou...


In [23]:
# vectorize
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_features = 50, 
                             ngram_range=(1, 3), 
                             stop_words='english',
                             binary=True)
X_train_counts = count_vect.fit(train_dropped['content'])
#X_train_counts.shape
X_train_counts = count_vect.transform(train_dropped['content'])
names =  count_vect.get_feature_names()
names[115:145]

# vectorize with tf-idf 
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print X_train_tf.shape

names = count_vect.get_feature_names()
names[50:100]

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print X_train_tfidf.shape

# use Naive-Bayes Classifier 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, train_dropped['article_subtopic'])

# Test the classifier 
docs_new = ['Business strategy is important.', 
            "Hiring new employees is tough because candidates are hard to find.", 
            "Candidates hire new employees", 
            "Web SEO mobile",
           "Attracting new customers through advertising",
           "Employee benefits",
           "Economic Ups and Downs",
           "Ups and Downs",
           "Customer stuff",
           "mobile",
           "digital",
           "The Power of Non-Linear Thinking",
           "3 Growth Strategies for Businesses With Limited"]
X_new_counts = count_vect.transform(docs_new)

X_new_tf = tf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, category))

(17, 50)
(17, 50)
'Business strategy is important.' => Taxes
'Hiring new employees is tough because candidates are hard to find.' => Strategy
'Candidates hire new employees' => Strategy
'Web SEO mobile' => Operations
'Attracting new customers through advertising' => Strategy
'Employee benefits' => Operations
'Economic Ups and Downs' => Operations
'Ups and Downs' => Operations
'Customer stuff' => Operations
'mobile' => Operations
'digital' => Operations
'The Power of Non-Linear Thinking' => Operations
'3 Growth Strategies for Businesses With Limited' => Operations


In [24]:
test_dropped = data_test[data_test['article_subtopic'].isin(categories)]
test_dropped.head(2)

,article_description,article_pubtime,article_section,article_subtopic,article_tags,article_topic,author_name,content,title
30,"As your business grows, you may want to consid...",2016-11-08T17:30:05+00:00,Managing Money,Cash Flow,"[Accounting, Cash Flow, Managing Money]",Managing Money,Hal Shelton,"When It Comes to Accounting, Do You Need a CPA...","When It Comes to Accounting, Do You Need a CPA..."
0,If you're having trouble trying to be more inn...,2016-11-11T20:00:15+00:00,Planning for Growth,Growth Opportunities,"[Growth Opportunities, Innovation, Personal Pr...",Planning for Growth,Heather Willems,The Power of Non-Linear ThinkingHeather Willem...,The Power of Non-Linear Thinking


In [25]:
text_clf = text_clf.fit(train_dropped['content'], train_dropped['article_subtopic'])

# get prediction accuracy using the test set
docs_test = test_dropped['content']
predicted = text_clf.predict(docs_test)
#print "Predicted:", predicted
np.mean(predicted == test_dropped['article_subtopic']) 
#test_dropped['article_subtopic']

0.15384615384615385

My accuracy slightly increased.. Promising! LOL.

The issues are:
1) there isn't enough data, and
2) there are too many buckets and the articles are not distributed evenly amongst them

### Attempting again
 Improvements:
 1) using article_topic instead of article_subtopic to have less buckets
 2) increase train set size 

In [26]:
# split data into train and test sets
from sklearn.cross_validation import train_test_split
topic_train, topic_test = train_test_split(df, test_size=0.18, random_state=42)
print "Train set len:", len(topic_train)
print "Test set len:", len(topic_test)

topic_train.head()
topic_labels = topic_train['article_topic']
print "Train Labels \n", topic_labels.value_counts()

topic_labels_test = topic_test['article_topic']
print "Test Labels \n", topic_labels_test.value_counts()

Train set len: 68
Test set len: 15
Train Labels 
Building Your Team     24
Getting Customers      24
Planning for Growth    12
Managing Money          8
Name: article_topic, dtype: int64
Test Labels 
Planning for Growth    7
Managing Money         4
Building Your Team     2
Getting Customers      2
Name: article_topic, dtype: int64


In [27]:
# vectorize
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_features = 100, 
                             ngram_range=(1, 3), 
                             stop_words='english',
                             binary=False
                            )
X_train_counts = count_vect.fit(topic_train['content'])
#X_train_counts.shape
X_train_counts = count_vect.transform(topic_train['content'])
names =  count_vect.get_feature_names()

# view snapshot of some the words found
names[0:10]

[u'000',
 u'alignment',
 u'alignment center',
 u'american',
 u'american express',
 u'article',
 u'articles',
 u'ask',
 u'author',
 u'based']

In [28]:
# vectorize with tf-idf 
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

names = count_vect.get_feature_names()
print names[50:100]

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print X_train_tfidf.shape

[u'media', u'money', u'need', u'new', u'open', u'owner', u'owners', u'pay', u'people', u'percent', u'photo', u'plan', u'process', u'product', u'products', u'professional', u'pullquote', u'pullquote showtweet', u'pullquote showtweet false', u'read', u'read articles', u'sales', u'says', u'self', u'service', u'showtweet', u'showtweet false', u'small', u'small business', u'social', u'social media', u'start', u'tax', u'team', u'theyre', u'things', u'think', u'time', u'twitter', u'use', u'using', u'want', u'way', u'ways', u'website', u'work', u'working', u'year', u'years', u'youre']
(68, 100)


In [29]:
# use Naive-Bayes Classifier 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, topic_labels)

In [30]:
# Test the classifier (article_topics version)
docs_new = ['Business strategy is important.', 
            "Hiring new employees is tough because candidates are hard to find.", 
            "Candidates hire new employees", 
            "Web SEO mobile strategy",
           "Attracting new customers through advertising",
           "Employee benefits",
           "Economic Ups and Downs",
           "Ups and Downs",
           "Customers are happy when you do this",
           "what"]
X_new_counts = count_vect.transform(docs_new)
X_new_tf = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, category))
        
print "\n Predict Probabilities:"        
print clf.predict_proba(X_new_tf)        

'Business strategy is important.' => Building Your Team
'Hiring new employees is tough because candidates are hard to find.' => Building Your Team
'Candidates hire new employees' => Building Your Team
'Web SEO mobile strategy' => Building Your Team
'Attracting new customers through advertising' => Getting Customers
'Employee benefits' => Building Your Team
'Economic Ups and Downs' => Building Your Team
'Ups and Downs' => Building Your Team
'Customers are happy when you do this' => Getting Customers
'what' => Building Your Team

 Predict Probabilities:
[[ 0.36921214  0.28836502  0.12046992  0.22195292]
 [ 0.59999164  0.12582559  0.06349306  0.2106897 ]
 [ 0.59999164  0.12582559  0.06349306  0.2106897 ]
 [ 0.35294118  0.35294118  0.11764706  0.17647059]
 [ 0.12534883  0.61058736  0.01719338  0.24687043]
 [ 0.35294118  0.35294118  0.11764706  0.17647059]
 [ 0.35294118  0.35294118  0.11764706  0.17647059]
 [ 0.35294118  0.35294118  0.11764706  0.17647059]
 [ 0.09381467  0.70651951  0.01317

In [31]:
docs_test = topic_test['content']
docs_test = 

SyntaxError: invalid syntax (<ipython-input-31-c25cb3b50a80>, line 2)

In [32]:
# Test the classifier (article_topics version)
docs_new = topic_test['content']
X_new_counts = count_vect.transform(docs_new)
X_new_tf = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tf)

label_and_pred = pd.DataFrame(predicted)
pd.concat([pd.DataFrame(predicted), topic_labels_test])
    
pd.DataFrame({ 'Predicted Value' : predicted, 'Actual Value': topic_labels_test})    

,Actual Value,Predicted Value
30,Managing Money,Managing Money
0,Planning for Growth,Building Your Team
22,Planning for Growth,Planning for Growth
31,Managing Money,Managing Money
18,Planning for Growth,Planning for Growth
28,Managing Money,Planning for Growth
10,Planning for Growth,Planning for Growth
53,Getting Customers,Getting Customers
4,Planning for Growth,Planning for Growth
12,Planning for Growth,Building Your Team


In [33]:
# clf.score(X_new_tf, topic_labels_test)  # .615
clf.score(X_new_tf, topic_labels_test)

0.66666666666666663

### Try a different classifier - KNN

In [34]:
# KNN with tf only
from sklearn.neighbors import KNeighborsClassifier

count_vect = CountVectorizer(max_features = 100, # only consider the top max_features ordered by term frequency across the corpus
                             ngram_range=(1, 3), # a , a b, a b c
                             stop_words='english',
                             binary=False # leave numerical values
                            )  

X_train_counts = count_vect.fit_transform(topic_train['content'])

X = X_train_counts  # vectorized content
y = topic_labels
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X, y)

X_test = tf_transformer.transform(count_vect.transform(topic_test['content']))
y_test = topic_labels_test
neigh.score(X_test, y_test)

0.13333333333333333

In [35]:
# KNN with tf-idf
tfidf_transformer = TfidfTransformer(use_idf=True)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X = X_train_tfidf  # vectorized content
y = topic_labels
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X, y)

X_test = tf_transformer.transform(count_vect.transform(topic_test['content']))
y_test = topic_labels_test
neigh.score(X_test, y_test)

0.53333333333333333

Modifying the vectorizer helped a lot. And TF-IDF definitely helps for normalization. 
But still not great.

### Cross Validation

In [36]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(MultinomialNB(), X, y, cv=8)
print "Scores:", scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores: [ 0.55555556  0.66666667  0.77777778  0.77777778  0.75        0.75        0.625
  0.75      ]
Accuracy: 0.71 (+/- 0.15)


In [37]:
# Cross-val on the whole dataset 
X = count_vect.fit_transform(df['content'])
X = tfidf_transformer.fit_transform(X)
y = df['article_topic']

scores = cross_val_score(MultinomialNB(), X, y, cv=8)
print "Scores:", scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores: [ 0.76923077  0.61538462  0.54545455  0.7         0.88888889  0.77777778
  0.88888889  0.88888889]
Accuracy: 0.76 (+/- 0.25)


In [38]:
scores = cross_val_score(KNeighborsClassifier(n_neighbors=4), X, y, cv=8)
print "Scores:", scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores: [ 0.69230769  0.53846154  0.63636364  0.6         0.66666667  0.88888889
  0.77777778  0.66666667]
Accuracy: 0.68 (+/- 0.20)


#### Random Forest


In [46]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=100), X, y, cv=8)
print "Scores:", scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores: [ 0.84615385  0.38461538  0.72727273  0.5         0.77777778  0.55555556
  0.77777778  0.77777778]
Accuracy: 0.67 (+/- 0.31)


### Gensim
